Вопросы для повторения:
* Что такое стэк и куча?
* Что такое указатель?
* `sizeof(int *)`?
* `sizeof(int)`?
* `sizeof(uint64_t)`?
* Куда указывает `p`?

    ```c++
    std::uint64_t arr[100];
    std::uint64_t *p = &arr[0];
    p = p + 8;
    ```
    
* Где хранятся данные строк при исполнении программы?

```c++
const char* s = "hello world";

std::string s2 = "London is the capital of Great Britain";
```

<br />

##### Эхо прошлого занятия: аргументы командной строки

Общепринят формат передачи аргументов командной строки в программу, в котором аргументы делятся на:

* позиционные
* флажки
* именованные со значением

Пример консольной утилиты linux `ls`, выводящей содержимое папки (по возможности продемонстрировать):

```
ls /usr/bin -a --human-readable --color=always
ls -a --color always --human-readable /usr/bin
```

описание опций можно прочитать, набрав `ls --help`:

```
  -a, --all                  do not ignore entries starting with .
  -h, --human-readable       with -l and/or -s, print human readable sizes
                               (e.g., 1K 234M 2G)
      --color[=WHEN]         colorize the output; WHEN can be 'always' (default
                               if omitted), 'auto', or 'never'; more info below
```

<br />

**Задача**: вывести аргументы командной строки, переданные консольной утилите

```c++
#include <iostream>

int main(int argc, char** argv)
{
    std::cout << "command line arguments are:" << std::endl;
    for (int i = 0; i < argc; ++i)
    {
        std::cout << "    " << i << " -> " << argv[i] << std::endl;
    }
    return 0;
}
```

Скомпилируем программу в исполняемый файл `a.out` и запустим: <br />
_(инструкции компиляции можно найти в лекции 1)_

Команда:
```
./a.out /usr/bin -a --human-readable --color=always
```

Вывод:
```
command line arguments are:
    0 -> ./a.out
    1 -> /usr/bin
    2 -> -a
    3 -> --human-readable
    4 -> --color=always
```

Команда:
```
./a.out /usr/bin -a --human-readable --color always
```

Вывод:
```
command line arguments are:
    0 -> ./a.out
    1 -> /usr/bin
    2 -> -a
    3 -> --human-readable
    4 -> --color
    5 -> always
```

Команда:
```
./a.out "/usr/bin/my awesome app dir" -a --human-readable --color=always
```
    
Вывод:
```
command line arguments are:
    0 -> ./a.out
    1 -> /usr/bin/my awesome app dir
    2 -> -a
    3 -> --human-readable
    4 -> --color=always
```

<br />

Для разбора аргументов командной строки используют готовые решения:
* C++, cross-platform [`boost::program_options`](https://www.boost.org/doc/libs/1_58_0/doc/html/program_options.html)
* C, unix-only [`getopt`](https://www.gnu.org/software/libc/manual/html_node/Example-of-Getopt.html)
* ...

<br />

<br />

### Лекция 2. Функции, пространства имён, заголовочные файлы

<br />

**Примечание:** ниже в примерах используется тип `std::vector<T>`. Если не встречались с ним раньше, то воспринимайте `std::vector` как динамический массив в куче. Т.е. он при необходимости увеличивает свой размер.

<br />

##### Функции, объявление и определение (declaration / definition)

Определение (definition) функции - описание её "интерфейса" (сигнатуры, возвращаемого типа и квалификаторов) И реализации.

```c++
float abs(float x)
{
    if (x >= 0)
        return x;
    return -x;
}

float min_value(const std::vector<float>& items)
{
    float rv = items.front();
    for (float x : items)
        rv = std::min(rv, x);
    return rv;
}
```

Замечание:
* для поиска минимума, конечно же, используйте `std::min_element`
* а для вычисления абсолютных величин функцию `abs` из `#include <cmath>`

Объявление (declaration) функции - описание её "интерфейса"

```c++
float abs(float x);

float min_value(const std::vector<float>& items);
```

Если не прописано явно, у функции в программе может быть несколько объявлений, но не больше одного определения.

__Вопрос__: что помещается в header-файл, что в cpp-файл?

<br />

##### Передача аргументов в функцию

Передача по значению - создание копии аргумента

```c++
float min_value(std::vector<float> x);
```

Передача по ссылке - работа с аргументом

```c++
float min_value(std::vector<float>& x);
```

Передача по const-ссылке - работа с аргументом и запрет на модификацию

```c++
float min_value(const std::vector<float>& x);
```

**Упражнение:** что здесь происходит с аргументами?

```c++
float sqr(const float x);

float min_value(std::vector<float>* x);
float min_value(const std::vector<float>* x);
float min_value(std::vector<float>* const x);
```

**Вопрос:** Когда лучше передавать по значению, а когда - по ссылке?

<br />

##### Возвращаемое значение

Функция ничего не возвращает:

```c++
void say_hello(const std::string& name)
{
    std::cout << "hello, " << name;
}
```

Возврат результата через возвращаемое значение (предпочтительный вариант):

```c++
std::vector<std::string> make_team()
{
    return { "Bifur", "Bofur", "Bombur", "Oin",
             "Gloin", "Fili", "Nori", "Dori",
             "Dwalin", "Ori", "Balin", "Kili" };
}
```

Возврат результата через аргумент (менее предпочтительный вариант в силу меньшей читаемости):

```c++
bool append_teamlead(Point3D location, std::vector<std::string>& team)
{
    if (is_shire(location))
    {
        team.push_back("Thorin");
        return true;
    }
    return false;
}
```

Сравните:
    
```c++
// легко читается, что есть результат функции
std::vector<std::string> team = make_team();

// не очевидно, что результатом функции является
// изменение второго аргумента, нужно лезть в
// документацию или реализацию, чтобы понять
// замысел автора
append_teamlead(get_current_location(), team);

```

<br />

##### Ошибки при работе с аргументами и возвращаемыми значениями

```c++
std::string* find_dwarf(const std::vector<std::string>& team, const std::string& name)
{
    for (const std::string& dwarf : team)
        if (dwarf == name)
            return &dwarf;
    return nullptr;
}

// usage 1
std::vector<std::string> team = make_team();
if (std::string* balin_ptr = find_dwarf(team, "Kuzya"))
    std::cout << *balin_ptr;

// usage 2
if (std::string* balin_ptr = find_dwarf(make_team(), "Balin"))
    std::cout << *balin_ptr;  // OOOPS

// usage 3
if (std::string* balin_ptr = find_dwarf({"Ori", "Gloin", "Balin"}, "Balin"))
    std::cout << *balin_ptr;  // OOOPS
```

__Вопрос__: что будет напечатано программой ниже?

Показать пример на godbolt.org на clang 8.0.0 с разными оптимизациями, чтобы наглядно продемонстрировать ub

```c++
#include <iostream>
#include <string>

const std::string& f(const bool x,
                     const std::string& s1,
                     const std::string& s2)
{
    return x ? s1 : s2;
}

int main()
{
    const std::string& s = f(true, "123", "12345");
    std::cout << s << endl;
    return 0;
}
```

<br />

##### Значения аргументов по умолчанию

Можно задавать значения аргументов по умолчанию:

```c++
std::string convert_to_string(int value, int base = 10);
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n");
```

использование:

```c++
std::string s1 = convert_to_string(42);    // 42
std::string s2 = convert_to_string(42, 2); // 101010

std::string song = join_strings({"In the town where I was born",
                                 "Lived a man who sailed to sea",
                                 "And he told us of his life",
                                 "In the land of submarines"});
std::string sentence = join_strings({"Run", "Forest", "run"}, " ");
```

Но такие аргументы должны быть последними:

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines);  // compilation ERROR!
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines = false);  // OK
```

<br />

##### Перегрузка функции

https://en.cppreference.com/book/intro/function_overloading

Задача - реализовать конвертацию всего в строку. Желательно единообразно и чтобы: есть способ - компилируется и работает, нет способа - не компилируется.

```c++
std::string convert_to_string(int x);       // 1
std::string convert_to_string(unsigned x);  // 2
std::string convert_to_string(float x);     // 3

std::cout << convert_to_string(5);    // 1
std::cout << convert_to_string(5u);   // 2
std::cout << convert_to_string(5.f);  // 3
```

<br />

##### Назначение header-файлов и cpp-файлов

Файлы в С++-программах делятся на 2 типа: компилируемые и включаемые:
    
* _Компилируемые_ файлы (`file.cpp`) - файлы с текстом на языке С++, подаются на вход компилятору для генерации соответствующего объектного файла с машинными интсрукциями (`file.o`)
* _Включаемые_ файлы (`file.h`) - файлы с текстом на языке С++, не являются конечной целью компиляции, их содержимое копируется в `cpp`-файлы при их компиляции через директиву `#include`

**Пример:** 

Напишем программу, по целому числу `n` считающую $(1 + 1/n)^n$.

Программа будет состоять из трёх файлов:
* `exp.cpp` - дефиниция (реализация) функции расчёта
* `exp.h` - декларация функции расчёта
* `prg.cpp` - дефиниция `main`

_Подробно объяснить программу_

Файл `exp.cpp`:

```c++
#include <cmath>

double approximate_exp(const unsigned int n)
{
    if (n == 0)
        return 1.0;
    
    return std::pow(1.0 + 1.0 / n, n);
}
```

Файл `exp.h`:

```c++
#pragma once

double approximate_exp(const unsigned int n);
```

Файл `prg.cpp`:

```c++
#include <iostream>

#include "exp.h"

int main()
{
    unsigned n = 0;
    std::cin >> n;
    
    std::cout << approximate_exp(n);    
    return 0;
}
```

Команды компиляции и линковки:
    
```bash
clang++ -c exp.cpp -o exp.o  # скомпилировать объектный файл exp.o с кодом вычислений
clang++ -c prg.cpp -o prg.o  # скомпилировать объектный файл prg.o с кодом запуска
clang++ exp.o prg.o -o prg.exe  # слинковать всё вместе в исполняемый файл prg.exe
```

<br />

---

Материал ниже ушёл на следующую лекцию

---

##### Пространства имён и name mangling при компиляции

https://en.wikipedia.org/wiki/Name_mangling

Пространства времён решают проблему, когда функция с одинаковым именем имеет две реализации в разных билиотеках

```c++
// library for json parsing

// return version of json parsing library
std::string get_version();
```

```c++
// library for image manipulation

// return version of image manipulation library
std::string get_version();
```

Такой код скомпилируется, но не слинкуется (либо слинкуется в некорректную программу), т.к. для функции `get_version` нарушено ODR-правило (One Definition Rule)

```c++
// library for json parsing

namespace json_parser
{
    // return version of json parsing library
    std::string get_version();
}
```

```c++
// library for image manipulation

namespace image_processing
{
   // return version of image manipulation library
    std::string get_version(); 
}
```

Такой код слинкуется корректно, т.к. работает механизм name mangling во время компиляции

Показать пример на godbolt.org, не забыть убрать галочку demangle

```c++
#include <iostream>
#include <string>

std::string get_version(int x, int y, long z);

namespace json_parser
{
    std::string get_version(int x, int y, long z);
}

namespace image_processing
{
    std::string get_version(int x, int y, long z);
}

int main()
{
    std::cout << get_version(0, 0, 0);
    std::cout << json_parser::get_version(0, 0, 0);
    std::cout << image_processing::get_version(0, 0, 0);
    return 0;
}
```

В результате name mangling компилятор clang создаёт символы:
* `_Z11get_versionB5cxx11iil`
* `_ZN16json_parser11get_versionB5cxx11Eiil`
* `_ZN21image_processing11get_versionB5cxx11Eiil`

Какой из них к чему относится, догадаться не сложно. 

<br />

##### Поиск функции и `using namespace`

Точные правила поиска:

https://en.cppreference.com/w/cpp/language/lookup

При компиляции куска кода:

```c++
namespace json_parser {
namespace input_processing {

int read_int(const std::string& s)
{
    if (avx_instructions_available())
        return read_int_avx(s);
    else
        return read_int_default(s);
}
    
}
}
```

Компилятору нужно найти кандидатов для вызова `avx_instructions_available`, `read_int_avx` и `read_int_default`. Компилятор осуществляет их поиск в пространствах имён:
* глобальное
* `json_parser`
* `json_parser::input_processing`
* пространства имён аргументов функций (ничего для `avx_instructions_available`, `std` для `read_int_avx` и `read_int_default`)

Если для какой-нибудь из функций находится либо более одного кандидата для вызова либо ни одного - ошибка компиляции. Кандидат должен быть и быть только один.

<br />

`using namespace X` - заполнить текущее пространство имён до закрываеющей скобки именами из пространства X.

**Пример 1:**

Вызывающая программа:

```c++
#include "json_parser.h"

int main()
{
    // не скомпилируется, т.к. в глобальном пространстве имён нет функции read_int
    std::cout << read_int(s);
    
    // скомпилируется
    std::cout << json_parser::input_processing::read_int(s);
    
    // скомпилируется, т.к. глобальное пространство имён расширено
    using namespace json_parser::input_processing;
    std::cout << read_int(s);
    
    return 0;    
}
```

**Пример 2:**

```c++
int main() {
    std::cout << 1 << std::string("23");
    return 0;
}
```

или

```c++
using namespace std; 
// имена из std доступны в глобальном пространстве имён до конца файла

int main() {
    cout << 1 << string("23");
    return 0;
}
```

или

```c++
int main()
{
    using namespace std; 
    // имена из std доступны в глобальном пространстве имён до конца main

    cout << 1 << string("23");
    return 0;
}
```

<br />

**Рекомендации:**

1. Никогда не пишите `using namespace` в `.h` - файлах (почему?)
2. Ограничивайте область действия `using namespace` рационально.

**Пример рациональности:** функция, считающая время выполнения другой функции (многословность `std::chrono`):

```c++
// без using namespace
std::uint64_t get_execution_time_microseconds(std::function<void> f)
{
    const std::chrono::high_resolution_clock::time_point start_ts =
        std::chrono::high_resolution_clock::now();
    f();    
    const std::chrono::high_resolution_clock::time_point final_ts =
        std::chrono::high_resolution_clock::now();    
    return std::chrono::duration_cast<std::chrono::microseconds>(final_ts - start_ts).count();
}

// с using namespace
std::uint64_t get_execution_time_microseconds(std::function<void> f)
{
    using namespace std::chrono;    
    const high_resolution_clock::time_point start_ts = high_resolution_clock::now();
    f();    
    const high_resolution_clock::time_point final_ts = high_resolution_clock::now();    
    return duration_cast<microseconds>(final_ts - start_ts).count();
}
```

<br />

**Замечания после лекции:**

* не успели поговорить про namespace
* т.к. на прошлом семинаре не разобрали пример с `read_n` и потоками, чуть не забыл рассказать про них в домашнем задании
* в домашнем задании надо рисовать как считать 32-битную сумму, как правило, этого не поимают
* пример про headers вызывает вопросы, т.к. в примере понятно как, но не понятно зачем. Нужно подобрать пример, чтобы без разбиения на h-cpp программа не скомпилировалась
* раздел про ошибки при работе с аргументами воспринимается тяжело. Надо его иллюстрировать ярче или переработать.
* пример про чтение файла блоками с семинара выкинуть, он скорее отчасти вреден чем полезен
* убрать using из примеров юниттестов